In [14]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

# 1. Create the model and memory:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
memory = ConversationBufferMemory(return_messages=True)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


# 2. Create the tools:
tools = [get_word_length]

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 3. Create the Prompt:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events and isn't good at calculating word length.",
        ),
        # This is where the agent will write/read its messages from
        MessagesPlaceholder(variable_name="agent_scratchpad"),
        MessagesPlaceholder(variable_name="history"),
        ("user", "{input}"),
    ]
)

In [16]:
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.runnables import RunnableLambda
from operator import itemgetter

# 4. Formats the python function tools into JSON schema and binds them to the model:
llm_with_tools = model.bind_tools(tools=[convert_to_openai_tool(t) for t in tools])

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

# 5. Setting up the agent chain:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "history": RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [17]:
# Construct the OpenAI functions agent:
agent = create_openai_tools_agent(llm_with_tools, tools, prompt)

In [18]:
# Create an agent executor by passing in the agent, tools and memory:
memory = ConversationBufferMemory(return_messages=True)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [19]:
agent_executor.invoke({"input": "My name is James"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'James'}`


5
Invoking: `get_word_length` with `{'word': 'My'}`


2
Invoking: `get_word_length` with `{'word': 'My'}`


2
Invoking: `get_word_length` with `{'word': 'name'}`


4
Invoking: `get_word_length` with `{'word': 'is'}`


2
Invoking: `get_word_length` with `{'word': 'James'}`


5The word lengths in the sentence "My name is James" are as follows:
- "My": 2
- "name": 4
- "is": 2
- "James": 5

> Finished chain.


{'input': 'My name is James',
 'history': [HumanMessage(content='My name is James'),
  AIMessage(content='The word lengths in the sentence "My name is James" are as follows:\n- "My": 2\n- "name": 4\n- "is": 2\n- "James": 5')],
 'output': 'The word lengths in the sentence "My name is James" are as follows:\n- "My": 2\n- "name": 4\n- "is": 2\n- "James": 5'}

In [20]:
agent_executor.invoke({"input": "What are the first three characters of my name?"})



> Entering new AgentExecutor chain...
The first three characters of your name "James" are "Jam".

> Finished chain.


{'input': 'What are the first three characters of my name?',
 'history': [HumanMessage(content='My name is James'),
  AIMessage(content='The word lengths in the sentence "My name is James" are as follows:\n- "My": 2\n- "name": 4\n- "is": 2\n- "James": 5'),
  HumanMessage(content='What are the first three characters of my name?'),
  AIMessage(content='The first three characters of your name "James" are "Jam".')],
 'output': 'The first three characters of your name "James" are "Jam".'}